In [42]:
library(anytime)
library(dplyr)
library(lubridate)
library(chron)

Warning message:
"package 'chron' was built under R version 3.4.3"
Attaching package: 'chron'

The following objects are masked from 'package:lubridate':

    days, hours, minutes, seconds, years



## 1 Beijing data
### 1.1Append AQ data

In [2]:
# read data: AQ (2 files)
bj_aq_a <- read.csv('Datasets/beijing_17_18_aq.csv')
bj_aq_b <- read.csv('Datasets/beijing_201802_201803_aq.csv')

In [3]:
# converting time to POSIXct format for easy transformations
bj_aq_a$utc_time <- anytime(bj_aq_a$utc_time)
bj_aq_b$utc_time <- anytime(bj_aq_b$utc_time)

bj_aq_a %>% summarize(min_date = min(utc_time), max_date = max(utc_time))
bj_aq_b %>% summarize(min_date = min(utc_time), max_date = max(utc_time))

min_date,max_date
2017-01-01 14:00:00,2018-01-31 15:00:00


min_date,max_date
2018-01-31 16:00:00,2018-03-31 15:00:00


In [4]:
# browse data
str(bj_aq_a)
str(bj_aq_b)

head(bj_aq_a)
head(bj_aq_b)

'data.frame':	311010 obs. of  8 variables:
 $ stationId: Factor w/ 35 levels "aotizhongxin_aq",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ utc_time : POSIXct, format: "2017-01-01 14:00:00" "2017-01-01 15:00:00" ...
 $ PM2.5    : num  453 417 395 420 453 429 211 116 51 38 ...
 $ PM10     : num  467 443 467 484 520 NA NA NA NA NA ...
 $ NO2      : num  156 143 141 139 157 141 110 87 58 55 ...
 $ CO       : num  7.2 6.8 6.9 7.4 7.6 6.5 3.3 2.2 1.3 1.1 ...
 $ O3       : num  3 2 3 3 4 3 NA 4 26 28 ...
 $ SO2      : num  9 8 8 9 9 9 11 13 14 14 ...
'data.frame':	49420 obs. of  8 variables:
 $ stationId: Factor w/ 35 levels "aotizhongxin_aq",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ utc_time : POSIXct, format: "2018-01-31 16:00:00" "2018-01-31 17:00:00" ...
 $ PM2.5    : num  49 47 46 60 52 38 30 29 26 28 ...
 $ PM10     : num  82 80 91 95 91 80 70 75 79 95 ...
 $ NO2      : num  90 90 91 85 81 72 70 73 73 73 ...
 $ CO       : num  0.9 0.9 1.3 2 1.9 1.2 0.9 0.8 0.9 1.1 ...
 $ O3       : num  6 5 5 6 5 4 3 3 4 7 ...

stationId,utc_time,PM2.5,PM10,NO2,CO,O3,SO2
aotizhongxin_aq,2017-01-01 14:00:00,453,467,156,7.2,3,9
aotizhongxin_aq,2017-01-01 15:00:00,417,443,143,6.8,2,8
aotizhongxin_aq,2017-01-01 16:00:00,395,467,141,6.9,3,8
aotizhongxin_aq,2017-01-01 17:00:00,420,484,139,7.4,3,9
aotizhongxin_aq,2017-01-01 18:00:00,453,520,157,7.6,4,9
aotizhongxin_aq,2017-01-01 19:00:00,429,NA,141,6.5,3,9


stationId,utc_time,PM2.5,PM10,NO2,CO,O3,SO2
aotizhongxin_aq,2018-01-31 16:00:00,49,82,90,0.9,6,10
aotizhongxin_aq,2018-01-31 17:00:00,47,80,90,0.9,5,10
aotizhongxin_aq,2018-01-31 18:00:00,46,91,91,1.3,5,28
aotizhongxin_aq,2018-01-31 19:00:00,60,95,85,2.0,6,38
aotizhongxin_aq,2018-01-31 20:00:00,52,91,81,1.9,5,30
aotizhongxin_aq,2018-01-31 21:00:00,38,80,72,1.2,4,14


In [5]:
# Append 2 AQ data sets
bj_aq <- rbind(bj_aq_a, bj_aq_b)
str(bj_aq)
bj_aq %>% summarize(min_date = min(utc_time), max_date = max(utc_time))

'data.frame':	360430 obs. of  8 variables:
 $ stationId: Factor w/ 35 levels "aotizhongxin_aq",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ utc_time : POSIXct, format: "2017-01-01 14:00:00" "2017-01-01 15:00:00" ...
 $ PM2.5    : num  453 417 395 420 453 429 211 116 51 38 ...
 $ PM10     : num  467 443 467 484 520 NA NA NA NA NA ...
 $ NO2      : num  156 143 141 139 157 141 110 87 58 55 ...
 $ CO       : num  7.2 6.8 6.9 7.4 7.6 6.5 3.3 2.2 1.3 1.1 ...
 $ O3       : num  3 2 3 3 4 3 NA 4 26 28 ...
 $ SO2      : num  9 8 8 9 9 9 11 13 14 14 ...


min_date,max_date
2017-01-01 14:00:00,2018-03-31 15:00:00


### 1.2 Prepare mapping table and grid data

In [6]:
# read data: map, GM
bj_closest_stations <- read.csv('SL_beijing_closest_stations.csv')
bj_grid_meo <- read.csv('Datasets/Beijing_historical_meo_grid.csv')

In [7]:
# Converting to POSIXct format
bj_grid_meo$utc_time <- anytime(bj_grid_meo$utc_time)
bj_grid_meo %>% summarize(min_date = min(utc_time), max_date = max(utc_time))

min_date,max_date
2017-01-01,2018-03-27 05:00:00


In [8]:
colnames(bj_closest_stations) <- c('x',"stationId","stationName","distance") 

In [9]:
str(bj_closest_stations)
str(bj_grid_meo)

head(bj_closest_stations)
head(bj_grid_meo)

'data.frame':	35 obs. of  4 variables:
 $ x          : int  1 2 3 4 5 6 7 8 9 10 ...
 $ stationId  : Factor w/ 35 levels "aotizhongxin_aq",..: 7 24 11 27 1 19 26 3 35 10 ...
 $ stationName: Factor w/ 26 levels "beijing_grid_216",..: 16 16 13 16 17 19 14 9 8 13 ...
 $ distance   : num  3540 1669 4638 4776 2020 ...
'data.frame':	7034706 obs. of  9 variables:
 $ stationName   : Factor w/ 651 levels "beijing_grid_000",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ longitude     : num  115 115 115 115 115 115 115 115 115 115 ...
 $ latitude      : num  39 39.1 39.2 39.3 39.4 39.5 39.6 39.7 39.8 39.9 ...
 $ utc_time      : POSIXct, format: "2017-01-01 00:00:00" "2017-01-01 00:00:00" ...
 $ temperature   : num  -5.47 -5.53 -5.7 -5.88 -5.34 -4.81 -4.98 -5.49 -6.17 -7.17 ...
 $ pressure      : num  985 979 963 947 929 ...
 $ humidity      : num  76.6 75.4 71.8 68.2 58.8 ...
 $ wind_direction: num  53.71 43.59 0.97 327.65 317.85 ...
 $ wind_speed.kph: num  3.53 3.11 2.75 3.84 6.14 ...


x,stationId,stationName,distance
1,dongsi_aq,beijing_grid_303,3539.569
2,tiantan_aq,beijing_grid_303,1669.215
3,guanyuan_aq,beijing_grid_282,4637.889
4,wanshouxigong_aq,beijing_grid_303,4775.641
5,aotizhongxin_aq,beijing_grid_304,2020.020
6,nongzhanguan_aq,beijing_grid_324,5296.386


stationName,longitude,latitude,utc_time,temperature,pressure,humidity,wind_direction,wind_speed.kph
beijing_grid_000,115,39.0,2017-01-01,-5.47,984.73,76.60,53.71,3.53
beijing_grid_001,115,39.1,2017-01-01,-5.53,979.33,75.40,43.59,3.11
beijing_grid_002,115,39.2,2017-01-01,-5.70,963.14,71.80,0.97,2.75
beijing_grid_003,115,39.3,2017-01-01,-5.88,946.94,68.20,327.65,3.84
beijing_grid_004,115,39.4,2017-01-01,-5.34,928.80,58.81,317.85,6.14
beijing_grid_005,115,39.5,2017-01-01,-4.81,910.66,49.43,313.46,8.52


### 1.3 Merge AQ and Mapping table

In [10]:
# STEP 1: Merge AQ and map
bj_aq_map <- merge(bj_aq,bj_closest_stations, by = "stationId")

### 1.4 Merge with GM

In [11]:
# STEP 2: Merge Step1 and GM
bj_aq_gm <- merge(bj_aq_map, bj_grid_meo, by = c("stationName","utc_time"))

In [12]:
str(bj_aq_gm)
head(bj_aq_gm)
bj_aq_gm %>% summarize(min_date = min(utc_time), max_date = max(utc_time))

'data.frame':	356825 obs. of  18 variables:
 $ stationName   : Factor w/ 26 levels "beijing_grid_216",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ utc_time      : POSIXct, format: "2017-01-01 14:00:00" "2017-01-01 15:00:00" ...
 $ stationId     : Factor w/ 35 levels "aotizhongxin_aq",..: 14 14 14 14 14 14 14 14 14 14 ...
 $ PM2.5         : num  376 369 361 354 356 315 287 254 231 224 ...
 $ PM10          : num  447 407 389 NA 360 NA NA NA NA NA ...
 $ NO2           : num  116 133 134 137 137 135 133 123 119 118 ...
 $ CO            : num  5.5 5.6 5.7 5.9 6.2 6.2 5.9 5.6 5.5 5.5 ...
 $ O3            : num  2 2 2 2 2 2 NA 2 2 2 ...
 $ SO2           : num  16 17 18 17 16 14 14 13 12 11 ...
 $ x             : int  30 30 30 30 30 30 30 30 30 30 ...
 $ distance      : num  2226 2226 2226 2226 2226 ...
 $ longitude     : num  116 116 116 116 116 116 116 116 116 116 ...
 $ latitude      : num  39.6 39.6 39.6 39.6 39.6 39.6 39.6 39.6 39.6 39.6 ...
 $ temperature   : num  -2.22 -2.37 -2.58 -2.79 -2.99 -3.3 -3

stationName,utc_time,stationId,PM2.5,PM10,NO2,CO,O3,SO2,x,distance,longitude,latitude,temperature,pressure,humidity,wind_direction,wind_speed.kph
beijing_grid_216,2017-01-01 14:00:00,liulihe_aq,376,447,116,5.5,2,16,30,2226.39,116,39.6,-2.22,1014.70,62.93,284.29,4.28
beijing_grid_216,2017-01-01 15:00:00,liulihe_aq,369,407,133,5.6,2,17,30,2226.39,116,39.6,-2.37,1014.86,65.78,297.82,4.39
beijing_grid_216,2017-01-01 16:00:00,liulihe_aq,361,389,134,5.7,2,18,30,2226.39,116,39.6,-2.58,1014.99,65.35,308.15,4.19
beijing_grid_216,2017-01-01 17:00:00,liulihe_aq,354,NA,137,5.9,2,17,30,2226.39,116,39.6,-2.79,1015.13,64.93,319.12,4.14
beijing_grid_216,2017-01-01 18:00:00,liulihe_aq,356,360,137,6.2,2,16,30,2226.39,116,39.6,-2.99,1015.26,64.50,329.98,4.24
beijing_grid_216,2017-01-01 19:00:00,liulihe_aq,315,NA,135,6.2,2,14,30,2226.39,116,39.6,-3.30,1015.53,64.80,324.88,4.10


min_date,max_date
2017-01-01 14:00:00,2018-03-27 05:00:00


### 1.5 Select columns and output CSV

In [13]:
# Select columns
bj_aq_gm <- bj_aq_gm[,c("stationName","utc_time","stationId","PM2.5","PM10","O3","temperature","pressure","humidity","wind_direction","wind_speed.kph")]
head(bj_aq_gm)

stationName,utc_time,stationId,PM2.5,PM10,O3,temperature,pressure,humidity,wind_direction,wind_speed.kph
beijing_grid_216,2017-01-01 14:00:00,liulihe_aq,376,447,2,-2.22,1014.70,62.93,284.29,4.28
beijing_grid_216,2017-01-01 15:00:00,liulihe_aq,369,407,2,-2.37,1014.86,65.78,297.82,4.39
beijing_grid_216,2017-01-01 16:00:00,liulihe_aq,361,389,2,-2.58,1014.99,65.35,308.15,4.19
beijing_grid_216,2017-01-01 17:00:00,liulihe_aq,354,NA,2,-2.79,1015.13,64.93,319.12,4.14
beijing_grid_216,2017-01-01 18:00:00,liulihe_aq,356,360,2,-2.99,1015.26,64.50,329.98,4.24
beijing_grid_216,2017-01-01 19:00:00,liulihe_aq,315,NA,2,-3.30,1015.53,64.80,324.88,4.10


**I am overriding Michelle's csv file just to avoid confusions on which file to use**

In [14]:
write.csv(bj_aq_gm, file = "SL_bj_aq_gm_historical.csv")

## 2 London Data
### 2.1 Append AQ data 

In [15]:
# read data: AQ
# Had some weird data in other stations dataset I downloaded - so added cleaning up logic too
ld_aq1 <- read.csv('Datasets/London_historical_aqi_forecast_stations_20180331.csv',stringsAsFactors=FALSE)
ld_aq2 <- read.csv('Datasets/London_historical_aqi_other_stations_20180331.csv',stringsAsFactors=FALSE)
ld_aq2 <- ld_aq2[,1:5]
ld_aq2 <- ld_aq2[!(ld_aq2$MeasurementDateGMT==''),]

In [16]:
ld_aq1$MeasurementDateGMT <-as.POSIXct(strptime(ld_aq1$MeasurementDateGMT, tz= "GMT", format = "%Y/%m/%d %H:%M" ))
ld_aq2$MeasurementDateGMT <-as.POSIXct(strptime(ld_aq2$MeasurementDateGMT, tz= "GMT", format = "%Y/%m/%d %H:%M" ))

In [17]:
# browse data
str(ld_aq1)
str(ld_aq2)

head(ld_aq1)
head(ld_aq2)

'data.frame':	141661 obs. of  6 variables:
 $ X                 : int  0 1 2 3 4 5 6 7 8 9 ...
 $ MeasurementDateGMT: POSIXct, format: "2017-01-01 00:00:00" "2017-01-01 01:00:00" ...
 $ station_id        : chr  "CD1" "CD1" "CD1" "CD1" ...
 $ PM2.5..ug.m3.     : num  40 31.6 24.7 21.2 24.9 24.6 23.9 22 19 19.9 ...
 $ PM10..ug.m3.      : num  44.4 34.4 28.1 24.5 23 23.9 22 22.9 20.1 24.4 ...
 $ NO2..ug.m3.       : num  36.6 46.2 38.3 32.8 28.1 29.3 28.8 34.6 44.6 55.3 ...
'data.frame':	118675 obs. of  5 variables:
 $ Station_ID        : chr  "LH0" "LH0" "LH0" "LH0" ...
 $ MeasurementDateGMT: POSIXct, format: "2017-01-01 00:00:00" "2017-01-01 01:00:00" ...
 $ PM2.5..ug.m3.     : num  30.2 25.4 24.7 23.6 24.2 22.8 21.6 19.9 18.3 16.3 ...
 $ PM10..ug.m3.      : num  34.6 29.2 28.1 27 27.4 26 24.8 23.1 21.3 19.5 ...
 $ NO2..ug.m3.       : num  15.9 11.8 11.6 13 27.1 22.9 26.8 39.4 41.6 44.1 ...


X,MeasurementDateGMT,station_id,PM2.5..ug.m3.,PM10..ug.m3.,NO2..ug.m3.
0,2017-01-01 00:00:00,CD1,40.0,44.4,36.6
1,2017-01-01 01:00:00,CD1,31.6,34.4,46.2
2,2017-01-01 02:00:00,CD1,24.7,28.1,38.3
3,2017-01-01 03:00:00,CD1,21.2,24.5,32.8
4,2017-01-01 04:00:00,CD1,24.9,23.0,28.1
5,2017-01-01 05:00:00,CD1,24.6,23.9,29.3


Station_ID,MeasurementDateGMT,PM2.5..ug.m3.,PM10..ug.m3.,NO2..ug.m3.
LH0,2017-01-01 00:00:00,30.2,34.6,15.9
LH0,2017-01-01 01:00:00,25.4,29.2,11.8
LH0,2017-01-01 02:00:00,24.7,28.1,11.6
LH0,2017-01-01 03:00:00,23.6,27.0,13.0
LH0,2017-01-01 04:00:00,24.2,27.4,27.1
LH0,2017-01-01 05:00:00,22.8,26.0,22.9


In [18]:
ld_aq1 %>% summarize(min_date = min(MeasurementDateGMT), max_date = max(MeasurementDateGMT))
ld_aq2 %>% summarize(min_date = min(MeasurementDateGMT), max_date = max(MeasurementDateGMT))

min_date,max_date
2017-01-01,2018-03-31


min_date,max_date
2017-01-01,2018-04-01


In [19]:
unique(ld_aq1$station_id, incomparables = FALSE)
unique(ld_aq2$Station_ID, incomparables = FALSE)

[1] "CD1" "BL0" "GR4" "MY7" "HV1" "GN3" "GR9" "LW2" "GN0" "KF1" "CD9" "ST5"
[13] "TH4"

[1] "LH0" "HR1" "TD5" "CT3" "GB0" "CR8" "RB7" "BX1" "BX9" "KC1" "CT2" ""

In [20]:
# Set to same column names
colnames(ld_aq1) <- c('x','utc_time','station_id','PM25','PM10','NO2')
colnames(ld_aq2) <- c('station_id','utc_time','PM25','PM10','NO2')

In [21]:
# Select relevant columns
ld_aq1 <- ld_aq1[,c('utc_time','station_id','PM25','PM10','NO2')]
ld_aq2 <- ld_aq2[,c('utc_time','station_id','PM25','PM10','NO2')]
head(ld_aq1)
head(ld_aq2)

utc_time,station_id,PM25,PM10,NO2
2017-01-01 00:00:00,CD1,40.0,44.4,36.6
2017-01-01 01:00:00,CD1,31.6,34.4,46.2
2017-01-01 02:00:00,CD1,24.7,28.1,38.3
2017-01-01 03:00:00,CD1,21.2,24.5,32.8
2017-01-01 04:00:00,CD1,24.9,23.0,28.1
2017-01-01 05:00:00,CD1,24.6,23.9,29.3


utc_time,station_id,PM25,PM10,NO2
2017-01-01 00:00:00,LH0,30.2,34.6,15.9
2017-01-01 01:00:00,LH0,25.4,29.2,11.8
2017-01-01 02:00:00,LH0,24.7,28.1,11.6
2017-01-01 03:00:00,LH0,23.6,27.0,13.0
2017-01-01 04:00:00,LH0,24.2,27.4,27.1
2017-01-01 05:00:00,LH0,22.8,26.0,22.9


In [22]:
# Append 2 AQ data sets
ld_aq <- rbind(ld_aq1, ld_aq2)
str(ld_aq)
ld_aq %>% summarize(min_date = min(utc_time), max_date = max(utc_time))

'data.frame':	260336 obs. of  5 variables:
 $ utc_time  : POSIXct, format: "2017-01-01 00:00:00" "2017-01-01 01:00:00" ...
 $ station_id: chr  "CD1" "CD1" "CD1" "CD1" ...
 $ PM25      : num  40 31.6 24.7 21.2 24.9 24.6 23.9 22 19 19.9 ...
 $ PM10      : num  44.4 34.4 28.1 24.5 23 23.9 22 22.9 20.1 24.4 ...
 $ NO2       : num  36.6 46.2 38.3 32.8 28.1 29.3 28.8 34.6 44.6 55.3 ...


min_date,max_date
2017-01-01,2018-04-01


### 2.2 Prepare mapping table and grid data

In [23]:
ld_closest_stations <- read.csv('SL_london_closest_stations.csv')
ld_grid_meo <- read.csv('Datasets/London_historical_meo_grid.csv')

In [24]:
# Converting to POSIXct format
ld_grid_meo$utc_time <- anytime(ld_grid_meo$utc_time)
ld_grid_meo %>% summarize(min_date = min(utc_time), max_date = max(utc_time))

min_date,max_date
2017-01-01,2018-03-27 05:00:00


In [25]:
colnames(ld_closest_stations) <- c('x',"station_id","stationName","distance")

In [26]:
# browse data
str(ld_closest_stations)
str(ld_grid_meo)

head(ld_closest_stations)
head(ld_grid_meo)

'data.frame':	24 obs. of  4 variables:
 $ x          : int  1 2 3 4 5 6 7 8 9 10 ...
 $ station_id : Factor w/ 24 levels "BL0","BX1","BX9",..: 3 2 1 5 4 7 8 6 10 12 ...
 $ stationName: Factor w/ 10 levels "london_grid_346",..: 10 10 6 6 4 6 6 5 8 8 ...
 $ distance   : num  3929 3929 3060 3683 5212 ...
'data.frame':	9303966 obs. of  9 variables:
 $ stationName   : Factor w/ 861 levels "london_grid_000",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ longitude     : num  -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 ...
 $ latitude      : num  50.5 50.6 50.7 50.8 50.9 51 51.1 51.2 51.3 51.4 ...
 $ utc_time      : POSIXct, format: "2017-01-01 00:00:00" "2017-01-01 00:00:00" ...
 $ temperature   : num  9.36 9.09 8.3 7.5 6.92 6.33 6.13 6.12 6.19 6.43 ...
 $ pressure      : num  1025 1024 1023 1021 1016 ...
 $ humidity      : num  77.9 79 82.1 85.3 88.5 ...
 $ wind_direction: num  251 250 245 236 228 ...
 $ wind_speed.kph: num  23.7 21.8 16.1 10.6 11 ...


x,station_id,stationName,distance
1,BX9,london_grid_472,3929.170
2,BX1,london_grid_472,3929.170
3,BL0,london_grid_409,3059.760
4,CD9,london_grid_409,3682.914
5,CD1,london_grid_388,5211.633
6,CT2,london_grid_409,1646.945


stationName,longitude,latitude,utc_time,temperature,pressure,humidity,wind_direction,wind_speed.kph
london_grid_000,-2,50.5,2017-01-01,9.36,1024.81,77.90,250.88,23.74
london_grid_001,-2,50.6,2017-01-01,9.09,1024.25,78.96,249.86,21.81
london_grid_002,-2,50.7,2017-01-01,8.30,1022.55,82.13,245.35,16.08
london_grid_003,-2,50.8,2017-01-01,7.50,1020.86,85.30,235.98,10.55
london_grid_004,-2,50.9,2017-01-01,6.92,1015.89,88.51,228.21,10.97
london_grid_005,-2,51.0,2017-01-01,6.33,1010.92,91.73,221.12,11.57


### 2.3 Merge AQ and mapping table

In [27]:
# STEP 1: Merge AQ and map
ld_aq_map <- merge(ld_aq,ld_closest_stations, by = "station_id")

In [28]:
head(ld_aq_map)
str(ld_aq_map)
summary(ld_aq_map)

station_id,utc_time,PM25,PM10,NO2,x,stationName,distance
BL0,2017-01-01 01:00:00,22.9,34.6,17.3,3,london_grid_409,3059.76
BL0,2017-01-01 02:00:00,18.4,25.2,14.2,3,london_grid_409,3059.76
BL0,2017-01-01 03:00:00,19.5,24.0,14.8,3,london_grid_409,3059.76
BL0,2017-01-01 00:00:00,30.8,31.6,10.8,3,london_grid_409,3059.76
BL0,2017-01-01 05:00:00,22.8,27.3,16.2,3,london_grid_409,3059.76
BL0,2017-01-01 06:00:00,19.6,25.3,19.5,3,london_grid_409,3059.76


'data.frame':	260335 obs. of  8 variables:
 $ station_id : chr  "BL0" "BL0" "BL0" "BL0" ...
 $ utc_time   : POSIXct, format: "2017-01-01 01:00:00" "2017-01-01 02:00:00" ...
 $ PM25       : num  22.9 18.4 19.5 30.8 22.8 19.6 15.6 18.1 14.5 10.9 ...
 $ PM10       : num  34.6 25.2 24 31.6 27.3 25.3 24 23.4 22.5 19.8 ...
 $ NO2        : num  17.3 14.2 14.8 10.8 16.2 19.5 21 28 26.4 25.3 ...
 $ x          : int  3 3 3 3 3 3 3 3 3 3 ...
 $ stationName: Factor w/ 10 levels "london_grid_346",..: 6 6 6 6 6 6 6 6 6 6 ...
 $ distance   : num  3060 3060 3060 3060 3060 ...


  station_id           utc_time                        PM25       
 Length:260335      Min.   :2017-01-01 00:00:00   Min.   :-14.00  
 Class :character   1st Qu.:2017-04-26 10:00:00   1st Qu.:  6.30  
 Mode  :character   Median :2017-08-17 10:00:00   Median : 10.00  
                    Mean   :2017-08-17 06:27:39   Mean   : 13.09  
                    3rd Qu.:2017-12-08 10:00:00   3rd Qu.: 16.00  
                    Max.   :2018-04-01 00:00:00   Max.   :313.00  
                                                  NA's   :49338   
      PM10             NO2               x                  stationName   
 Min.   :-11.80   Min.   : -8.30   Min.   : 1.00   london_grid_388:43611  
 1st Qu.: 11.20   1st Qu.: 19.50   1st Qu.: 7.00   london_grid_451:43611  
 Median : 16.80   Median : 34.20   Median :13.00   london_grid_409:42202  
 Mean   : 20.17   Mean   : 39.73   Mean   :12.53   london_grid_472:32737  
 3rd Qu.: 25.20   3rd Qu.: 53.80   3rd Qu.:19.00   london_grid_430:32691  
 Max.   :667.1

### 2.4 Merge with GM

In [29]:
# STEP 2: Merge Step1 and GM
ld_aq_gm <- merge(ld_aq_map, ld_grid_meo, by = c("stationName","utc_time"))

In [30]:
head(ld_aq_gm)
str(ld_aq_gm)
summary(ld_aq_gm)

stationName,utc_time,station_id,PM25,PM10,NO2,x,distance,longitude,latitude,temperature,pressure,humidity,wind_direction,wind_speed.kph
london_grid_346,2017-01-01 08:00:00,LH0,18.3,21.3,41.6,16,3143.783,-0.4,51.5,6.05,1019.42,90.04,221.29,16.38
london_grid_346,2017-01-01 09:00:00,LH0,16.3,19.5,44.1,16,3143.783,-0.4,51.5,6.05,1018.21,89.16,219.73,17.05
london_grid_346,2017-01-01 10:00:00,LH0,13.3,16.2,49.1,16,3143.783,-0.4,51.5,6.04,1017.00,88.28,218.29,17.73
london_grid_346,2017-01-01 11:00:00,LH0,9.4,11.6,45.2,16,3143.783,-0.4,51.5,6.04,1015.79,87.40,216.96,18.42
london_grid_346,2017-01-01 12:00:00,LH0,6.1,8.5,41.4,16,3143.783,-0.4,51.5,6.25,1014.82,87.85,215.76,18.82
london_grid_346,2017-01-01 13:00:00,LH0,6.7,13.4,53.6,16,3143.783,-0.4,51.5,6.46,1013.85,88.31,214.60,19.22


'data.frame':	257936 obs. of  15 variables:
 $ stationName   : Factor w/ 10 levels "london_grid_346",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ utc_time      : POSIXct, format: "2017-01-01 08:00:00" "2017-01-01 09:00:00" ...
 $ station_id    : chr  "LH0" "LH0" "LH0" "LH0" ...
 $ PM25          : num  18.3 16.3 13.3 9.4 6.1 6.7 2.1 0.9 1.1 1 ...
 $ PM10          : num  21.3 19.5 16.2 11.6 8.5 13.4 4.6 2 2.1 2 ...
 $ NO2           : num  41.6 44.1 49.1 45.2 41.4 53.6 11.7 12.1 12 13.5 ...
 $ x             : int  16 16 16 16 16 16 16 16 16 16 ...
 $ distance      : num  3144 3144 3144 3144 3144 ...
 $ longitude     : num  -0.4 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4 ...
 $ latitude      : num  51.5 51.5 51.5 51.5 51.5 51.5 51.5 51.5 51.5 51.5 ...
 $ temperature   : num  6.05 6.05 6.04 6.04 6.25 6.46 6.67 6.91 7.15 7.39 ...
 $ pressure      : num  1019 1018 1017 1016 1015 ...
 $ humidity      : num  90 89.2 88.3 87.4 87.8 ...
 $ wind_direction: num  221 220 218 217 216 ...
 $ wind_speed.kph: num  

          stationName       utc_time                    station_id       
 london_grid_388:43224   Min.   :2017-01-01 08:00:00   Length:257936     
 london_grid_451:43224   1st Qu.:2017-04-25 16:00:00   Class :character  
 london_grid_409:41816   Median :2017-08-15 15:00:00   Mode  :character  
 london_grid_430:32418   Mean   :2017-08-15 11:34:58                     
 london_grid_472:32418   3rd Qu.:2017-12-05 15:00:00                     
 london_grid_408:21612   Max.   :2018-03-27 12:00:00                     
 (Other)        :43224                                                   
      PM25             PM10             NO2               x        
 Min.   :-14.00   Min.   :-11.80   Min.   : -8.30   Min.   : 1.00  
 1st Qu.:  6.30   1st Qu.: 11.20   1st Qu.: 19.50   1st Qu.: 7.00  
 Median : 10.00   Median : 16.80   Median : 34.20   Median :13.00  
 Mean   : 13.13   Mean   : 20.23   Mean   : 39.72   Mean   :12.54  
 3rd Qu.: 16.00   3rd Qu.: 25.30   3rd Qu.: 53.80   3rd Qu.:19.00  


### 2.5 Select columns and output CSV

In [31]:
# Select columns
ld_aq_gm <- ld_aq_gm[,c("stationName","utc_time","station_id","PM25","PM10","NO2","temperature","pressure","humidity","wind_direction","wind_speed.kph")]
head(ld_aq_gm)

stationName,utc_time,station_id,PM25,PM10,NO2,temperature,pressure,humidity,wind_direction,wind_speed.kph
london_grid_346,2017-01-01 08:00:00,LH0,18.3,21.3,41.6,6.05,1019.42,90.04,221.29,16.38
london_grid_346,2017-01-01 09:00:00,LH0,16.3,19.5,44.1,6.05,1018.21,89.16,219.73,17.05
london_grid_346,2017-01-01 10:00:00,LH0,13.3,16.2,49.1,6.04,1017.00,88.28,218.29,17.73
london_grid_346,2017-01-01 11:00:00,LH0,9.4,11.6,45.2,6.04,1015.79,87.40,216.96,18.42
london_grid_346,2017-01-01 12:00:00,LH0,6.1,8.5,41.4,6.25,1014.82,87.85,215.76,18.82
london_grid_346,2017-01-01 13:00:00,LH0,6.7,13.4,53.6,6.46,1013.85,88.31,214.60,19.22


**Rewriting Michelle's files to avoid confusions**

In [32]:
write.csv(ld_aq_gm, file = "SL_ld_aq_gm_historical.csv")

## 3 Adding additional features

### London 

In [34]:
ld_aq_gm$hour <- hour(ld_aq_gm$utc_time)
ld_aq_gm$month <- month(ld_aq_gm$utc_time)
ld_aq_gm$date <- date(ld_aq_gm$utc_time)
ld_aq_gm$weekend = chron::is.weekend(ld_aq_gm$date)

In [46]:
str(ld_aq_gm)

'data.frame':	257936 obs. of  15 variables:
 $ stationName   : Factor w/ 10 levels "london_grid_346",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ utc_time      : POSIXct, format: "2017-01-01 08:00:00" "2017-01-01 09:00:00" ...
 $ station_id    : chr  "LH0" "LH0" "LH0" "LH0" ...
 $ PM25          : num  18.3 16.3 13.3 9.4 6.1 6.7 2.1 0.9 1.1 1 ...
 $ PM10          : num  21.3 19.5 16.2 11.6 8.5 13.4 4.6 2 2.1 2 ...
 $ NO2           : num  41.6 44.1 49.1 45.2 41.4 53.6 11.7 12.1 12 13.5 ...
 $ temperature   : num  6.05 6.05 6.04 6.04 6.25 6.46 6.67 6.91 7.15 7.39 ...
 $ pressure      : num  1019 1018 1017 1016 1015 ...
 $ humidity      : num  90 89.2 88.3 87.4 87.8 ...
 $ wind_direction: num  221 220 218 217 216 ...
 $ wind_speed.kph: num  16.4 17.1 17.7 18.4 18.8 ...
 $ hour          : int  8 9 10 11 12 13 14 15 16 17 ...
 $ month         : num  1 1 1 1 1 1 1 1 1 1 ...
 $ date          : Date, format: "2017-01-01" "2017-01-01" ...
 $ weekend       : logi  TRUE TRUE TRUE TRUE TRUE TRUE ...


**Rewriting Kelsey's file to avoid confusions**

In [47]:
write.csv(ld_aq_gm, file = "ld_aq_gm_hist_data.csv")

### Beijing

In [48]:
bj_aq_gm$hour <- hour(bj_aq_gm$utc_time)
bj_aq_gm$month <- month(bj_aq_gm$utc_time)
bj_aq_gm$date <- date(bj_aq_gm$utc_time)
bj_aq_gm$weekend = chron::is.weekend(bj_aq_gm$date)

In [49]:
str(bj_aq_gm)

'data.frame':	356825 obs. of  15 variables:
 $ stationName   : Factor w/ 26 levels "beijing_grid_216",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ utc_time      : POSIXct, format: "2017-01-01 14:00:00" "2017-01-01 15:00:00" ...
 $ stationId     : Factor w/ 35 levels "aotizhongxin_aq",..: 14 14 14 14 14 14 14 14 14 14 ...
 $ PM2.5         : num  376 369 361 354 356 315 287 254 231 224 ...
 $ PM10          : num  447 407 389 NA 360 NA NA NA NA NA ...
 $ O3            : num  2 2 2 2 2 2 NA 2 2 2 ...
 $ temperature   : num  -2.22 -2.37 -2.58 -2.79 -2.99 -3.3 -3.6 -3.9 -4.11 -4.33 ...
 $ pressure      : num  1015 1015 1015 1015 1015 ...
 $ humidity      : num  62.9 65.8 65.3 64.9 64.5 ...
 $ wind_direction: num  284 298 308 319 330 ...
 $ wind_speed.kph: num  4.28 4.39 4.19 4.14 4.24 4.1 3.99 3.92 2.97 2.25 ...
 $ hour          : int  14 15 16 17 18 19 20 21 22 23 ...
 $ month         : num  1 1 1 1 1 1 1 1 1 1 ...
 $ date          : Date, format: "2017-01-01" "2017-01-01" ...
 $ weekend       : logi  TR

**Rewriting Kelsey's file to avoid confusions**

In [50]:
write.csv(bj_aq_gm, file = "bj_aq_gm_hist_data.csv")